In [1]:
import time
import json
import os
import pandas as pd
import numpy as np
import flwr as fl
import pickle


from sklearn.model_selection import train_test_split

from hydra import initialize, compose
from omegaconf import OmegaConf, DictConfig

from logging import INFO, DEBUG
from flwr.common.logger import log


from src.models.evaluation_metrics import custom_acc_mc, custom_acc_binary
from src.data.dataset_info import datasets

with initialize(version_base=None, config_path="conf/"):
    cfg = compose(config_name='config.yaml')
    print(OmegaConf.to_yaml(cfg))


dataset = datasets[0]

folder_path = "./datasets/gdlc/"

learning_rate = 0.001
LAMBD_1 = 0.0001
LAMBD_2 = 0.001

multi_class: false
with_network_features: false
n_clients: 5
n_rounds: 20
config_fit:
  lr: 0.01
  momentum: 0.9
  local_epochs: 1
  batch_size: 256



In [2]:
dtime = time.strftime("%Y%m%d-%H%M%S")
dtime

'20240819-091455'

In [3]:
clients_paths = [
    folder_path + "client_0.parquet",
    folder_path + "client_1.parquet",
    folder_path + "client_2.parquet",
    folder_path + "client_3.parquet",
    folder_path + "client_4.parquet",
    folder_path + "client_5.parquet",
    folder_path + "client_6.parquet",
    folder_path + "client_7.parquet",
    folder_path + "test.parquet"
]

# clients_paths = [
#     folder_path + "client_0_pca.parquet",
#     folder_path + "client_1_pca.parquet",
#     folder_path + "client_2_pca.parquet",
#     folder_path + "client_3_pca.parquet",
#     folder_path + "client_4_pca.parquet",
#     folder_path + "client_5_pca.parquet",
#     folder_path + "client_6_pca.parquet",
#     folder_path + "client_7_pca.parquet",
#     folder_path + "test.parquet"
# ]

# clients_paths = [
#     folder_path + "client_0_global_pca.parquet",
#     folder_path + "client_1_global_pca.parquet",
#     folder_path + "client_2_global_pca.parquet",
#     folder_path + "client_3_global_pca.parquet",
#     folder_path + "client_4_global_pca.parquet",
#     folder_path + "client_5_global_pca.parquet",
#     folder_path + "client_6_global_pca.parquet",
#     folder_path + "client_7_global_pca.parquet",
#     folder_path + "test_global_pca.parquet"
# ]

# Data Loading and Preprocessing

In [4]:
with open(folder_path + "added_columns.pkl", 'rb') as f:
    centralities_columns, pca_columns = pickle.load(f)
    
centralities_columns, pca_columns

([['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_degree',
   'dst_degree',
   'src_local_degree',
   'dst_local_degree',
   'src_eigenvector',
   'dst_eigenvector',
   'src_closeness',
   'dst_closeness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',
   'src_Comm',
   'dst_Comm'],
  ['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',
   'src_Comm',
   'dst_Comm'],
  ['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',


In [5]:
test = pd.read_parquet(folder_path + "test.parquet")
print(test.columns)

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       ...
       'src_global_pagerank', 'dst_global_pagerank', 'src_k_core',
       'dst_k_core', 'src_k_truss', 'dst_k_truss', 'src_mv', 'dst_mv',
       'global_pca_1', 'global_pca_2'],
      dtype='object', length=109)


In [6]:
client_data = []
for client_path in clients_paths:
    client_data.append(pd.read_parquet(client_path))

In [7]:
client_columns = []

for client_df in client_data:
    client_columns.append(set(client_df.columns))

common_columns = set.intersection(*client_columns)

differences = [columns - common_columns for columns in client_columns]

# Display the columns of each client, the intersection, and the differences
#for idx, columns in enumerate(client_columns):
 #   print(f"Client {idx} columns: {columns}")

print(f"\nIntersection of columns across all clients: {common_columns}")

for idx, diff in enumerate(differences):
    print(f"Difference in columns for client {idx}: {diff}")


Intersection of columns across all clients: {'Src Port', 'Pkt Len Var', 'Bwd Header Len', 'Tot Fwd Pkts', 'Bwd IAT Std', 'TotLen Bwd Pkts', 'Active Mean', 'Subflow Bwd Pkts', 'Fwd IAT Min', 'Pkt Len Min', 'Fwd IAT Max', 'src_pagerank', 'Subflow Fwd Byts', 'Attack', 'Flow ID', 'Subflow Fwd Pkts', 'Tot Bwd Pkts', 'Bwd PSH Flags', 'Bwd IAT Mean', 'dst_k_truss', 'Timestamp', 'Down/Up Ratio', 'Fwd Act Data Pkts', 'Bwd Pkts/s', 'Class', 'Init Fwd Win Byts', 'Active Std', 'Bwd IAT Tot', 'Pkt Len Max', 'Fwd Pkts/s', 'Dst IP', 'Flow Pkts/s', 'URG Flag Cnt', 'Flow Duration', 'Fwd Blk Rate Avg', 'RST Flag Cnt', 'src_k_truss', 'Flow Byts/s', 'Bwd Blk Rate Avg', 'Fwd Seg Size Avg', 'src_betweenness', 'Flow IAT Mean', 'Pkt Len Std', 'Subflow Bwd Byts', 'global_pca_2', 'Flow IAT Min', 'Flow IAT Max', 'Bwd IAT Min', 'Pkt Size Avg', 'Init Bwd Win Byts', 'Flow IAT Std', 'Fwd URG Flags', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Src IP', 'Idle Min', 'dst_betweenness', 'CWE Flag Count', 'Fwd IAT Mean', 'Fwd

In [8]:
# the input dimension of the training set
# input_dim = df.shape[1] - len(drop_columns) - len(weak_columns) - 1  # for the label_column
  
# specifying the number of classes, since it is different from one dataset to another and also if binary or multi-class classification
classes_set = {"benign", "attack"}
labels_names = {0: "benign", 1: "attack"}
num_classes = 2
if cfg.multi_class:
    with open(folder_path + "labels_names.pkl", 'rb') as f:
        labels_names, classes_set = pickle.load(f)
    num_classes = len(classes_set)
    
labels_names = {int(k): v for k, v in labels_names.items()}

print(f"==>> classes_set: {classes_set}")
print(f"==>> num_classes: {num_classes}")
print(f"==>> labels_names: {labels_names}")

==>> classes_set: {'attack', 'benign'}
==>> num_classes: 2
==>> labels_names: {0: 'benign', 1: 'attack'}


In [9]:


if cfg.multi_class:
    test[dataset.label_col] = test[dataset.class_num_col]
    
    
test.drop(centralities_columns[-1], axis=1, inplace=True)
test.drop(pca_columns, axis=1, inplace=True)
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
# test.drop(["pca_1", "pca_2"], axis=1, inplace=True)
if not cfg.multi_class:
    test_by_class = {}
    classes = test[dataset.class_col].unique()
    for class_value in classes:
        test_class = test[test[dataset.class_col] == class_value].copy()
        test_class.drop(dataset.drop_columns, axis=1, inplace=True)
        test_class.drop(dataset.weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[dataset.label_col].to_numpy()
        test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)
    
    
test.drop(dataset.drop_columns, axis=1, inplace=True)
test.drop(dataset.weak_columns, axis=1, inplace=True)
test.reset_index(drop=True, inplace=True)
    
test_labels = test[dataset.label_col].to_numpy()
test = test.drop([dataset.label_col], axis=1).to_numpy()
input_dim = test.shape[1]
    
for i in range(len(client_data)):
    
    cdata = client_data[i]
    
    if cfg.multi_class:
        cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
    cdata.drop(centralities_columns[i], axis=1, inplace=True)
    cdata.drop(pca_columns, axis=1, inplace=True)
    # cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    # cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    cdata.drop(dataset.drop_columns, axis=1, inplace=True)
    cdata.drop(dataset.weak_columns, axis=1, inplace=True)
    cdata.reset_index(drop=True, inplace=True)
    
    c_train, c_test = train_test_split(cdata, test_size=0.1)

    y_train = c_train[dataset.label_col].to_numpy()
    x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
    y_test = c_test[dataset.label_col].to_numpy()
    x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
    client_data[i] = (x_train, y_train, x_test, y_test)

# Model

In [10]:
from keras import layers, models, Input, regularizers, callbacks, metrics, optimizers, initializers
# from src.models.evaluation_metrics import f1_m

def create_keras_model(input_shape, alpha = learning_rate):
    model = models.Sequential()
    
    model.add(layers.Conv1D(80, kernel_size=3,
                activation="relu", input_shape=(input_shape, 1), kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    # .L1L2(l1=LAMBD_1, l2=LAMBD_2)
    model.add(layers.Conv1D(80, 3, activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    
    # model.add(layers.LSTM(units=80,
    #                         activation='relu',
    #                         kernel_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         recurrent_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         bias_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         return_sequences=False,
    #                         ))
    # model.add(layers.LayerNormalization(axis=1))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(80,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))

    if cfg.multi_class:
        model.add(layers.Dense(num_classes, activation='softmax'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    else:
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    
    
    return model


In [11]:
model = create_keras_model(input_dim)
model.summary()

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 36, 80)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 18, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 18, 80)         │            36 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 16, 80)         │        19,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 8, 80)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 8, 80)          │            16 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       128,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, 80)             │           160 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,173 (801.46 KB)

 Trainable params: 205,173 (801.46 KB)

 Non-trainable params: 0 (0.00 B)

# FL

## FL settings

In [12]:
results_final = {}

results_final["model"] = {}

results_final["baseline"] = {}
results_final["baseline"]["accuracy"] = {}
results_final["baseline"]["f1s"] = {}

results_final["centralities - PCA"] = {}
results_final["centralities - PCA"]["accuracy"] = {}
results_final["centralities - PCA"]["f1s"] = {}

results_final["centralities - DiGraph"] = {}
results_final["centralities - DiGraph"]["accuracy"] = {}
results_final["centralities - DiGraph"]["f1s"] = {}

results_final["centralities - MultiDiGraph"] = {}
results_final["centralities - MultiDiGraph"]["accuracy"] = {}
results_final["centralities - MultiDiGraph"]["f1s"] = {}

In [13]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - baseline"
results["dtime"] = dtime
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

{'configuration': '2dt - baseline',
 'dtime': '20240819-091455',
 'multi_class': False,
 'learning_rate': 0.001,
 'dataset_name': 'cic_ton_iot',
 'num_classes': 2,
 'labels_names': {0: 'benign', 1: 'attack'},
 'input_dim': 38,
 'scores': {'server': {},
  'clients': {},
  'accuracy': {},
  'f1s': {},
  'test_by_class': {'accuracy': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hulk': {},
    'DoS GoldenEye': {},
    'PortScan': {},
    'DoS slowloris': {},
    'FTP-Patator': {},
    'SSH-Patator': {},
    'Bot': {},
    'DoS Slowhttptest': {},
    'bruteforce': {},
    'Infiltration': {},
    'Web Attack � Sql Injection': {},
    'Heartbleed': {}},
   'f1s': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hu

In [14]:

class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x_train, y_train, x_test, y_test):
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        self.model = create_keras_model(input_shape=input_dim, alpha=lr)
        self.set_parameters(parameters, config)

        
        logdir = "logs/scalars/{}/baseline/client_{}".format(dtime, self.cid)
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        history = self.model.fit(self.x_train, self.y_train,
                                 epochs=config["local_epochs"],
                                 batch_size=config["batch_size"],
                                 validation_data=(self.x_test, self.y_test),
                                 verbose=0,
                                 callbacks=[tensorboard_callback])

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [15]:
def generate_client_fn():
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

    return client_fn

In [16]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/baseline/server".format(dtime) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final["baseline"]["accuracy"][server_round] = scores["accuracy"]
        results_final["baseline"]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [17]:
def weighted_average(metrics):
    print(f"==>> weighted_average: {metrics}")
    results
    return metrics
    # total_examples = 0
    # federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    # for num_examples, m in metrics:
    #     for k, v in m.items():
    #         federated_metrics[k] += num_examples * v
    #     total_examples += num_examples
    # return {k: v / total_examples for k, v in federated_metrics.items()}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    # fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels),
)  # a function to run on the server side to evaluate the global model.

# strategy.aggregate_fit = 


## FL Simulation

In [18]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_cpus": 1,
        "num_gpus": 0.0,
    },
)

INFO flwr 2024-08-19 09:15:12,280 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-08-19 09:15:18,043	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-08-19 09:15:21,338 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 32.0, 'node:__internal_head__': 1.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 17675271782.0, 'memory': 35350543566.0}
INFO flwr 2024-08-19 09:15:21,339 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-08-19 09:15:21,340 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-08-19 09:15:21,371 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2024-08-19 09:15:21,372 | server.py:89 | Initializing global parameters
INFO flwr 2024-08-19 09:15:21,373 | server.py:276 | Requesting initial parameters from o

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.3768 - loss: 1.7175
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - accuracy: 0.7975 - loss: 0.5580
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 23s 13ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

...

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9643 - loss: 0.1233
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.9667 - loss: 0.1167
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

In [19]:
print(f"==>> history: {history}")
print(f"==>> end of history")

==>> history: History (loss, centralized):
	round 0: 1.688915491104126
	round 1: 0.5658921599388123
	round 2: 0.4416791498661041
	round 3: 0.45321565866470337
	round 4: 0.44293537735939026
	round 5: 0.38503435254096985
	round 6: 0.35959967970848083
	round 7: 0.2832050919532776
	round 8: 0.3070182204246521
	round 9: 0.2585664391517639
	round 10: 0.2014206498861313
	round 11: 0.14461107552051544
	round 12: 0.1701217144727707
	round 13: 0.23644548654556274
	round 14: 0.1885738968849182
	round 15: 0.1694631278514862
	round 16: 0.14690828323364258
	round 17: 0.12584418058395386
	round 18: 0.1482786387205124
	round 19: 0.16040095686912537
	round 20: 0.14419393241405487
History (metrics, centralized):
{'accuracy': [(0, 0.3832067847251892), (1, 0.7773797512054443), (2, 0.7966827154159546), (3, 0.8135286569595337), (4, 0.8111865520477295), (5, 0.8394028544425964), (6, 0.8598524332046509), (7, 0.9097967743873596), (8, 0.8958035111427307), (9, 0.9161360859870911), (10, 0.9274936318397522), (11, 0

In [20]:
# creating the directories if they don't exist
if not os.path.isdir('./results'):
    os.mkdir('./results')

# creating the directories if they don't exist
if not os.path.isdir('./results/{}'.format(dtime)):
    os.mkdir('./results/{}'.format(dtime))

# if not os.path.isdir('./results/{}'.format(dataset_name)):
#     os.mkdir('./results/{}'.format(dataset_name))

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

filename = ('./results/{}/baseline.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

In [21]:
filename = ('./results/{}/results_final.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
outfile.close()

# FL - PCA

In [22]:
test = pd.read_parquet(folder_path + "test.parquet")

if cfg.multi_class:
    test[dataset.label_col] = test[dataset.class_num_col]
    
test.drop(centralities_columns[-1], axis=1, inplace=True)
# test.drop(pca_columns, axis=1, inplace=True)
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
#test.drop(["pca_1", "pca_2"], axis=1, inplace=True)

if not cfg.multi_class:
    test_by_class = {}
    classes = test[dataset.class_col].unique()
    for class_value in classes:
        test_class = test[test[dataset.class_col] == class_value].copy()
        test_class.drop(dataset.drop_columns, axis=1, inplace=True)
        test_class.drop(dataset.weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[dataset.label_col].to_numpy()
        test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)
    
    
test.drop(dataset.drop_columns, axis=1, inplace=True)
test.drop(dataset.weak_columns, axis=1, inplace=True)
test.reset_index(drop=True, inplace=True)
    
test_labels = test[dataset.label_col].to_numpy()
test = test.drop([dataset.label_col], axis=1).to_numpy()
input_dim = test.shape[1]

client_data = []
for client_path in clients_paths:
    client_data.append(pd.read_parquet(client_path))
    
for i in range(len(client_data)):
    
    cdata = client_data[i]
    
    if cfg.multi_class:
        cdata[dataset.label_col] = cdata[dataset.class_num_col]
                
    cdata.drop(centralities_columns[i], axis=1, inplace=True)
    # cdata.drop(pca_columns, axis=1, inplace=True)
    
    # cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
   # cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    cdata.drop(dataset.drop_columns, axis=1, inplace=True)
    cdata.drop(dataset.weak_columns, axis=1, inplace=True)
    cdata.reset_index(drop=True, inplace=True)
    
    c_train, c_test = train_test_split(cdata, test_size=0.1)

    y_train = c_train[dataset.label_col].to_numpy()
    x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
    y_test = c_test[dataset.label_col].to_numpy()
    x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
    client_data[i] = (x_train, y_train, x_test, y_test)

In [23]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - PCA"
results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

{'configuration': '2dt - PCA',
 'dtime': '20240819-102404',
 'multi_class': False,
 'learning_rate': 0.001,
 'dataset_name': 'cic_ton_iot',
 'num_classes': 2,
 'labels_names': {0: 'benign', 1: 'attack'},
 'input_dim': 40,
 'scores': {'server': {},
  'clients': {},
  'accuracy': {},
  'f1s': {},
  'test_by_class': {'accuracy': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hulk': {},
    'DoS GoldenEye': {},
    'PortScan': {},
    'DoS slowloris': {},
    'FTP-Patator': {},
    'SSH-Patator': {},
    'Bot': {},
    'DoS Slowhttptest': {},
    'bruteforce': {},
    'Infiltration': {},
    'Web Attack � Sql Injection': {},
    'Heartbleed': {}},
   'f1s': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hulk': 

In [24]:
model = create_keras_model(input_dim)
model.summary()

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_44 (Conv1D)              │ (None, 38, 80)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_44 (MaxPooling1D) │ (None, 19, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_110         │ (None, 19, 80)         │            38 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_45 (Conv1D)              │ (None, 17, 80)         │        19,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_45 (MaxPooling1D) │ (None, 8, 80)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_111         │ (None, 8, 80)          │            16 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 200)            │       128,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_112         │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_89 (Dense)                │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_113         │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_90 (Dense)                │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_114         │ (None, 80)             │           160 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_91 (Dense)                │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,175 (801.46 KB)

 Trainable params: 205,175 (801.46 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:

class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x_train, y_train, x_test, y_test):
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
        self.model = create_keras_model(input_shape=input_dim, alpha=lr)
        # log(INFO, f"==>> config: {config}")
        # log(INFO, f"==>> float(config[lr]): {lr}")
        self.set_parameters(parameters, config)

        
        logdir = "logs/scalars/{}/PCA/client_{}".format(dtime, self.cid)
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        history = self.model.fit(self.x_train, self.y_train,
                                 epochs=config["local_epochs"],
                                 batch_size=config["batch_size"],
                                 validation_data=(self.x_test, self.y_test),
                                 verbose=0,
                                 callbacks=[tensorboard_callback])

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [26]:
def generate_client_fn():
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

    return client_fn

In [27]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/PCA/server".format(dtime) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final["centralities - PCA"]["accuracy"][server_round] = scores["accuracy"]
        results_final["centralities - PCA"]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [28]:
def weighted_average(metrics):
    print(f"==>> weighted_average: {metrics}")

    total_examples = 0
    federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    for num_examples, m in metrics:
        for k, v in m.items():
            federated_metrics[k] += num_examples * v
        total_examples += num_examples
    return {k: v / total_examples for k, v in federated_metrics.items()}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels),
)  # a function to run on the server side to evaluate the global model.


In [29]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_cpus": 1,
        "num_gpus": 0.0,
    },
)

INFO flwr 2024-08-19 10:24:04,544 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-08-19 10:24:15,832	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-08-19 10:24:18,466 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 32.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 17451128832.0, 'memory': 34902257664.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-08-19 10:24:18,467 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-08-19 10:24:18,468 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-08-19 10:24:18,495 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2024-08-19 10:24:18,496 | server.py:89 | Initializing global parameters
INFO flwr 2024-08-19 10:24:18,498 | server.py:276 | Requesting initial parameters from o

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.6297 - loss: 1.4524
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

==>> weighted_average: [(423074, {'accuracy': 0.9373064637184143, 'loss': 0.3657473921775818, 'val_accuracy': 0.9208661913871765, 'val_loss': 0.26545119285583496}), (489061, {'accuracy': 0.9513046145439148, 'loss': 0.3079336881637573, 'val_accuracy': 0.961944043636322, 'val_loss': 0.17417730391025543}), (489061, {'accuracy': 0.9855784773826599, 'loss': 0.25965437293052673, 'val_accuracy': 0.9996503591537476, 'val_loss': 0.06918796896934509}), (454118, {'accuracy': 0.9925063252449036, 'loss': 0.20739087462425232, 'val_accuracy': 0.997007429599762, 'val_loss': 0.06608185917139053}), (489061, {'accuracy': 0.9253610372543335, 'loss': 0.429437518119812, 'val_accuracy': 0.9730038046836853, 'val_loss': 0.17536944150924683}), (489061, {'accuracy': 0.9694189429283142, 'loss': 0.29746612906455994, 'val_accuracy': 0.9890137910842896, 'val_loss': 0.10686587542295456}), (454119, {'accuracy': 0.9957411885261536, 'loss': 0.16255436837673187, 'val_accuracy': 0.9997225403785706, 'val_loss': 0.022415112

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

...

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

==>> weighted_average: [(489061, {'accuracy': 0.9996912479400635, 'loss': 0.0032314497511833906, 'val_accuracy': 0.9999815821647644, 'val_loss': 0.0021042488515377045}), (489061, {'accuracy': 0.999492883682251, 'loss': 0.004098678007721901, 'val_accuracy': 0.9999632239341736, 'val_loss': 0.002176800975576043}), (489061, {'accuracy': 0.9993600249290466, 'loss': 0.004219698719680309, 'val_accuracy': 0.9999815821647644, 'val_loss': 0.002247051801532507}), (489061, {'accuracy': 0.9996033310890198, 'loss': 0.0033572702668607235, 'val_accuracy': 0.999944806098938, 'val_loss': 0.0023318848107010126}), (423074, {'accuracy': 0.9987897872924805, 'loss': 0.007152542006224394, 'val_accuracy': 0.9990427494049072, 'val_loss': 0.004785964731127024}), (454119, {'accuracy': 0.9999471306800842, 'loss': 0.002148990985006094, 'val_accuracy': 1.0, 'val_loss': 0.0018151550320908427}), (489061, {'accuracy': 0.9996605515480042, 'loss': 0.003533771960064769, 'val_accuracy': 0.9999815821647644, 'val_loss': 0.00

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

==>> weighted_average: [(423074, {'accuracy': 0.9990332722663879, 'loss': 0.005644579418003559, 'val_accuracy': 0.9996383786201477, 'val_loss': 0.0034965607337653637}), (489061, {'accuracy': 0.9998425841331482, 'loss': 0.002507549012079835, 'val_accuracy': 1.0, 'val_loss': 0.0018924233736470342}), (454119, {'accuracy': 0.9995441436767578, 'loss': 0.003881193930283189, 'val_accuracy': 0.9999603629112244, 'val_loss': 0.0022135009057819843}), (489061, {'accuracy': 0.9998160004615784, 'loss': 0.0024767538998275995, 'val_accuracy': 0.9997055530548096, 'val_loss': 0.00281127099879086}), (489061, {'accuracy': 0.9992393851280212, 'loss': 0.005030883010476828, 'val_accuracy': 0.999944806098938, 'val_loss': 0.002373232739046216}), (454118, {'accuracy': 0.9993591904640198, 'loss': 0.004736749455332756, 'val_accuracy': 0.9998414516448975, 'val_loss': 0.0029996554367244244}), (489061, {'accuracy': 0.9995890259742737, 'loss': 0.0036114684771746397, 'val_accuracy': 0.9996687769889832, 'val_loss': 0.0

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

In [30]:
print(f"==>> history: {history}")
print(f"==>> end of history")

==>> history: History (loss, centralized):
	round 0: 1.4819270372390747
	round 1: 1.4307628870010376
	round 2: 0.6223888993263245
	round 3: 0.5432789921760559
	round 4: 1.0336236953735352
	round 5: 0.9034113883972168
	round 6: 0.8937349915504456
	round 7: 1.1224803924560547
	round 8: 1.1490494012832642
	round 9: 1.2142587900161743
	round 10: 1.2597702741622925
	round 11: 0.6148693561553955
	round 12: 0.2778656482696533
	round 13: 0.6265143752098083
	round 14: 0.8682164549827576
	round 15: 0.1676424890756607
	round 16: 0.38637658953666687
	round 17: 0.19337446987628937
	round 18: 0.10652659088373184
	round 19: 0.3052070736885071
	round 20: 0.045693784952163696
History (metrics, distributed, fit):
{'accuracy': [(1, 0.9679665464383305), (2, 0.9734963352120666), (3, 0.9744727582477909), (4, 0.9562399951283759), (5, 0.9707273110275003), (6, 0.990463361728016), (7, 0.9937585241692188), (8, 0.9947103777665204), (9, 0.996697977959246), (10, 0.9972253004842734), (11, 0.998289417365025), (12, 0.

In [31]:
filename = ('./results/{}/pca.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

In [32]:
filename = ('./results/{}/results_final.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
outfile.close()

# Centralities - DiGraph

In [33]:
test = pd.read_parquet(folder_path + "test.parquet")

if cfg.multi_class:
    test[dataset.label_col] = test[dataset.class_num_col]
    
# test.drop(centralities_columns[-1], axis=1, inplace=True)
test.drop(pca_columns, axis=1, inplace=True)
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
# test.drop(["pca_1", "pca_2"], axis=1, inplace=True)

if not cfg.multi_class:
    test_by_class = {}
    classes = test[dataset.class_col].unique()
    for class_value in classes:
        test_class = test[test[dataset.class_col] == class_value].copy()
        test_class.drop(dataset.drop_columns, axis=1, inplace=True)
        test_class.drop(dataset.weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[dataset.label_col].to_numpy()
        test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)
    
    
test.drop(dataset.drop_columns, axis=1, inplace=True)
test.drop(dataset.weak_columns, axis=1, inplace=True)
test.reset_index(drop=True, inplace=True)
    
test_labels = test[dataset.label_col].to_numpy()
test = test.drop([dataset.label_col], axis=1).to_numpy()
input_dim = test.shape[1]

client_data = []
for client_path in clients_paths:
    client_data.append(pd.read_parquet(client_path))
    
for i in range(len(client_data)):
    
    cdata = client_data[i]
    
    if cfg.multi_class:
        cdata[dataset.label_col] = cdata[dataset.class_num_col]
                
    # cdata.drop(centralities_columns[i], axis=1, inplace=True)
    cdata.drop(pca_columns, axis=1, inplace=True)
    # cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    # cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    cdata.drop(dataset.drop_columns, axis=1, inplace=True)
    cdata.drop(dataset.weak_columns, axis=1, inplace=True)
    cdata.reset_index(drop=True, inplace=True)
    
    c_train, c_test = train_test_split(cdata, test_size=0.1)

    y_train = c_train[dataset.label_col].to_numpy()
    x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
    y_test = c_test[dataset.label_col].to_numpy()
    x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
    client_data[i] = (x_train, y_train, x_test, y_test)

In [35]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - Centralities - DiGraph"
results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

{'configuration': '2dt - Centralities - DiGraph',
 'dtime': '20240819-194141',
 'multi_class': False,
 'learning_rate': 0.001,
 'dataset_name': 'cic_ton_iot',
 'num_classes': 2,
 'labels_names': {0: 'benign', 1: 'attack'},
 'input_dim': 60,
 'scores': {'server': {},
  'clients': {},
  'accuracy': {},
  'f1s': {},
  'test_by_class': {'accuracy': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hulk': {},
    'DoS GoldenEye': {},
    'PortScan': {},
    'DoS slowloris': {},
    'FTP-Patator': {},
    'SSH-Patator': {},
    'Bot': {},
    'DoS Slowhttptest': {},
    'bruteforce': {},
    'Infiltration': {},
    'Web Attack � Sql Injection': {},
    'Heartbleed': {}},
   'f1s': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {

In [36]:
model = create_keras_model(input_dim)
model.summary()

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_44"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_88 (Conv1D)              │ (None, 58, 80)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_88 (MaxPooling1D) │ (None, 29, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_220         │ (None, 29, 80)         │            58 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_89 (Conv1D)              │ (None, 27, 80)         │        19,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_89 (MaxPooling1D) │ (None, 13, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_221         │ (None, 13, 80)         │            26 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_44 (Flatten)            │ (None, 1040)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_176 (Dense)               │ (None, 200)            │       208,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_222         │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_177 (Dense)               │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_223         │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_178 (Dense)               │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_224         │ (None, 80)             │           160 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_179 (Dense)               │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 285,205 (1.09 MB)

 Trainable params: 285,205 (1.09 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:

class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x_train, y_train, x_test, y_test):
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
        self.model = create_keras_model(input_shape=input_dim, alpha=lr)
        # log(INFO, f"==>> config: {config}")
        # log(INFO, f"==>> float(config[lr]): {lr}")
        self.set_parameters(parameters, config)

        
        logdir = "logs/scalars/{}/digraph/client_{}".format(dtime, self.cid)
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        history = self.model.fit(self.x_train, self.y_train,
                                 epochs=config["local_epochs"],
                                 batch_size=config["batch_size"],
                                 validation_data=(self.x_test, self.y_test),
                                 verbose=0,
                                 callbacks=[tensorboard_callback])

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [38]:
def generate_client_fn():
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

    return client_fn

In [39]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/digraph/server".format(dtime) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final["centralities - DiGraph"]["accuracy"][server_round] = scores["accuracy"]
        results_final["centralities - DiGraph"]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [40]:
def weighted_average(metrics):
    print(f"==>> weighted_average: {metrics}")

    total_examples = 0
    federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    for num_examples, m in metrics:
        for k, v in m.items():
            federated_metrics[k] += num_examples * v
        total_examples += num_examples
    return {k: v / total_examples for k, v in federated_metrics.items()}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels),
)  # a function to run on the server side to evaluate the global model.


In [41]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_cpus": 1,
        "num_gpus": 0.0,
    },
)

INFO flwr 2024-08-19 19:41:41,554 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-08-19 19:41:54,470	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-08-19 19:41:57,064 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 32.0, 'node:__internal_head__': 1.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 16973907148.0, 'memory': 33947814299.0}
INFO flwr 2024-08-19 19:41:57,065 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-08-19 19:41:57,067 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-08-19 19:41:57,097 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2024-08-19 19:41:57,099 | server.py:89 | Initializing global parameters
INFO flwr 2024-08-19 19:41:57,100 | server.py:276 | Requesting initial parameters from o

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.5776 - loss: 1.6253
1825/1837 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

In [ ]:
print(f"==>> history: {history}")
print(f"==>> end of history")

In [ ]:
filename = ('./results/{}/digraph.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

In [ ]:
filename = ('./results/{}/results_final.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
outfile.close()

# Centralities - MultiDiGraph

In [ ]:
# test = pd.read_parquet(folder_path + "test.parquet")

# if cfg.multi_class:
#     test[dataset.label_col] = test[dataset.class_num_col]
    
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# # test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)

# if not cfg.multi_class:
#     test_by_class = {}
#     classes = test[dataset.class_col].unique()
#     for class_value in classes:
#         test_class = test[test[dataset.class_col] == class_value].copy()
#         test_class.drop(dataset.drop_columns, axis=1, inplace=True)
#         test_class.drop(dataset.weak_columns, axis=1, inplace=True)
#         test_class.reset_index(drop=True, inplace=True)

#         test_class_labels = test_class[dataset.label_col].to_numpy()
#         test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

#         test_by_class[class_value] = (test_class, test_class_labels)
    
    
# test.drop(dataset.drop_columns, axis=1, inplace=True)
# test.drop(dataset.weak_columns, axis=1, inplace=True)
# test.reset_index(drop=True, inplace=True)
    
# test_labels = test[dataset.label_col].to_numpy()
# test = test.drop([dataset.label_col], axis=1).to_numpy()
# input_dim = test.shape[1]

# client_data = []
# for client_path in clients_paths:
#     client_data.append(pd.read_parquet(client_path))
    
# for i in range(len(client_data)):
    
#     cdata = client_data[i]
    
#     if cfg.multi_class:
#         cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
#     cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
#     # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)

#     cdata.drop(dataset.drop_columns, axis=1, inplace=True)
#     cdata.drop(dataset.weak_columns, axis=1, inplace=True)
#     cdata.reset_index(drop=True, inplace=True)
    
#     c_train, c_test = train_test_split(cdata, test_size=0.1)

#     y_train = c_train[dataset.label_col].to_numpy()
#     x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
#     y_test = c_test[dataset.label_col].to_numpy()
#     x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
#     client_data[i] = (x_train, y_train, x_test, y_test)

In [ ]:
# test.head()

In [ ]:
# results = {}  # a dictionary that will contain all the options and results of models
# # add all options to the results dictionary, to know what options selected for obtained results
# results["configuration"] = "2dt - Centralities - MultiDiGraph"
# results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
# results["multi_class"] = cfg.multi_class
# results["learning_rate"] = learning_rate
# results["dataset_name"] = dataset.name
# results["num_classes"] = num_classes
# results["labels_names"] = labels_names
# results["input_dim"] = input_dim

# results["scores"] = {}
# results["scores"]["server"] = {}
# results["scores"]["clients"] = {}
# results["scores"]["accuracy"] = {}
# results["scores"]["f1s"] = {}

# if not cfg.multi_class:
#     results["scores"]["test_by_class"] = {}
#     results["scores"]["test_by_class"]["accuracy"] = {}
#     results["scores"]["test_by_class"]["f1s"] = {}
#     for k in test_by_class.keys():
#         results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
#         results["scores"]["test_by_class"]["accuracy"][k] = {}   
#         results["scores"]["test_by_class"]["f1s"][k] = {}    
        
# results

In [ ]:
# model = create_keras_model(input_dim)
# model.summary()

In [ ]:

# class FLClient(fl.client.NumPyClient):
#     def __init__(self, cid, x_train, y_train, x_test, y_test):
#         self.cid = cid
#         self.x_train, self.y_train = x_train, y_train
#         self.x_test, self.y_test = x_test, y_test
#         self.model = create_keras_model(input_shape=input_dim)

#     def get_parameters(self, config):
#         return self.model.get_weights()

#     def set_parameters(self, parameters, config):
#         self.model.set_weights(parameters)

#     def fit(self, parameters, config):
        
#         lr=float(config["lr"])
#         # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
#         self.model = create_keras_model(input_shape=input_dim, alpha=lr)
#         # log(INFO, f"==>> config: {config}")
#         # log(INFO, f"==>> float(config[lr]): {lr}")
#         self.set_parameters(parameters, config)

        
#         logdir = "logs/scalars/{}/multidigraph/client_{}".format(dtime, self.cid)
#         tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

#         history = self.model.fit(self.x_train, self.y_train,
#                                  epochs=config["local_epochs"],
#                                  batch_size=config["batch_size"],
#                                  validation_data=(self.x_test, self.y_test),
#                                  verbose=0,
#                                  callbacks=[tensorboard_callback])

#         return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters, config)
#         loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
#         return loss, len(self.x_test), {"accuracy": accuracy}


In [ ]:
# def generate_client_fn():
#     def client_fn(cid: str):
#         i = int(cid)
#         return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

#     return client_fn

In [ ]:
# def get_on_fit_config(config: DictConfig):

#     def fit_config_fn(server_round: int):
#         alpha = learning_rate
#         if server_round > 5:
#             alpha = alpha / (1 + 0.5 * server_round)


#         return {
#             "lr": alpha,
#             "local_epochs": config.local_epochs,
#             "batch_size": config.batch_size,
#         }

#     return fit_config_fn


# def get_evaluate_fn(x_test_sever, y_test_server):

#     def evaluate_fn(server_round: int, parameters, config):
#         # eval_model = model
#         eval_model = create_keras_model(input_shape=input_dim)
#         eval_model.set_weights(parameters)

        
#         logdir = "logs/scalars/{}/multidigraph/server".format(dtime) 
#         # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
#         tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

#         test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
#                                                   batch_size = cfg.config_fit.batch_size,
#                                                   callbacks=[tensorboard_callback])
        
        
#         y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
#         if cfg.multi_class:
#             y_pred = np.argmax(y_pred, axis=1)
#             scores = custom_acc_mc(y_test_server, y_pred)
#         else:
#             y_pred = np.transpose(y_pred)[0]
#             y_pred = list(
#                 map(lambda x: 0 if x < 0.5 else 1, y_pred))
#             scores = custom_acc_binary(y_test_server, y_pred)
        
        
#         results["scores"]["accuracy"][server_round] = test_acc
#         results["scores"]["f1s"][server_round] = scores["f1s"]
#         results["scores"]["server"][server_round] = scores
        
        
#         results["scores"]["accuracy"][server_round] = test_acc
#         results["scores"]["f1s"][server_round] = scores["f1s"]
#         results["scores"]["server"][server_round] = scores
        
#         results_final["centralities - MultiDiGraph"]["accuracy"][server_round] = scores["accuracy"]
#         results_final["centralities - MultiDiGraph"]["f1s"][server_round] = scores["f1s"]
        
#         if not cfg.multi_class:
#             for k in test_by_class.keys():
#                 y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
#                 y_pred_class = np.transpose(y_pred_class)[0]
#                 y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
#                 scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
#                 results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
#                 results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
#         log(INFO, f"==>> scores: {scores}")
        
        
#         return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

#     return evaluate_fn


In [ ]:
# def weighted_average(metrics):
#     print(f"==>> weighted_average: {metrics}")

#     total_examples = 0
#     federated_metrics = {k: 0 for k in metrics[0][1].keys()}
#     for num_examples, m in metrics:
#         for k, v in m.items():
#             federated_metrics[k] += num_examples * v
#         total_examples += num_examples
#     return {k: v / total_examples for k, v in federated_metrics.items()}

# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
#     min_fit_clients=len(client_data),  # number of clients to sample for fit()
#     fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
#     min_evaluate_clients=0,  # number of clients to sample for evaluate()
#     min_available_clients=len(client_data),  # total clients in the simulation
#     fit_metrics_aggregation_fn = weighted_average,
#     # evaluate_metrics_aggregation_fn = weighted_average,
#     on_fit_config_fn=get_on_fit_config(
#         cfg.config_fit
#     ),  # a function to execute to obtain the configuration to send to the clients during fit()
#     evaluate_fn=get_evaluate_fn(test, test_labels),
# )  # a function to run on the server side to evaluate the global model.


In [ ]:
# import multiprocessing
# from math import floor
# history = fl.simulation.start_simulation(
#     client_fn=generate_client_fn(),  # a function that spawns a particular client
#     # num_clients=cfg.n_clients,  # total number of clients
#     num_clients=len(client_data),  # total number of clients
#     config=fl.server.ServerConfig(
#         num_rounds=cfg.n_rounds
#         # num_rounds=5
#     ),  # minimal config for the server loop telling the number of rounds in FL
#     strategy=strategy,  # our strategy of choice
#     client_resources={
#         "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
#         "num_gpus": 0.0,
#     },
# )

In [ ]:
# print(f"==>> history: {history}")
# print(f"==>> end of history")

In [ ]:
# filename = ('./results/{}/multidigraph.json'.format(dtime))
# outfile = open(filename, 'w')
# outfile.writelines(json.dumps(results, cls=NumpyEncoder))
# outfile.close()

In [ ]:
# filename = ('./results/{}/results_final.json'.format(dtime))
# outfile = open(filename, 'w')
# outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
# outfile.close()